In [1]:
# 1. Create Notebook and download libraries and packages
# import libraries
import pandas as pd # library to analyze data
import requests # library to handle web requests
from bs4 import BeautifulSoup
#
print('Libraries imported')

Libraries imported


In [2]:
# 2. Download wikipedia page, extract PostalCode, Burough & Neighbourhood table data & parse it
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(url.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [3]:
# 3. Create dataframe 
# 3.a) Show table columns table-headers <<table_h>> by removing <th> tags of HTML
table_h=table.find_all('th')
for i, head in enumerate(table_h): table_h[i]=str(table_h[i]).replace("\n","").replace("<th>","").replace("</th>","")
print(table_h) # view table_h output 

['Postcode', 'Borough', 'Neighbourhood']


In [4]:
# 3. a) continued...Build dataframe before removing Not assigned
#
#Find all table-rows tr & skip first item
table_r=table.find_all('tr')
table_r=table_r[1:len(table_r)]
#
# ignore symbols & line feeds in between rows by replacing td & tr with blank spaces
for i, row in enumerate(table_r): table_r[i] = str(table_r[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")   
#
# create a dataframe, but text in rows still contains '<a href=' but getting closer to final output
wikidf=pd.DataFrame(table_r)
# extract text from each column by removing </td>\n<td> and splitting into 3 columns
wikidf[table_h] = wikidf[0].str.split("</td>\n<td>", n = 2, expand = True) 
# drop first column of unwanted data in dataframe
wikidf.drop(columns=[0],inplace=True)
#
# Update dataframe by removing HTML tags & symbols & extract text of title from Burough & Neighbourhood columns
wikidf.update(
# look for string that contains 'title' in Burough column
    wikidf.Borough.loc[
        lambda B: B.str.contains('title')
# extract text from title in Burough column
    ].str.extract('title=\"([^\"]*)',expand=False))
#
# do the same for next column Neighbourhood 
wikidf.update(
    wikidf.Neighbourhood.loc[
        lambda N: N.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))
wikidf # The wiki page has 289 rows in the table, you can test this by exporting table to a spreadsheet

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
# 3. b) Only process cells that have an assigned Borough & ignore those Not assigned
#
# Drop table rows with Borough <<Not assigned>>
wikidf = wikidf.drop(wikidf[(wikidf.Borough == "Not assigned")].index)
# copy Neighbourhood value to Borough if cell value is Null Value
wikidf.Borough.fillna(wikidf.Neighbourhood, inplace=True)
#
# Cleanup unwanted duplicate text in columns
# remove <<(Toronto)>> value from Borough & Neighbourhood columns
wikidf.update(
    wikidf.Neighbourhood.loc[
        lambda NT: NT.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

wikidf.update(
    wikidf.Borough.loc[
        lambda BT: BT.str.contains('Toronto')
    ].str.replace(" \(Toronto\)",""))

# remove <<, Toronto>> value from Burough & Neighbourhood columns
wikidf.update(
    wikidf.Neighbourhood.loc[
        lambda NTX: NTX.str.contains('Toronto')
    ].str.replace(", Toronto",""))
#
wikidf.update(
    wikidf.Borough.loc[
        lambda BTX: BTX.str.contains('Toronto')
    ].str.replace(", Toronto",""))
#
wikidf.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [6]:
# 3. c) Combine Neighbourhoods with same PostalCode like M5A
#
# combine Neighborhoods with identical PostalCode
# create unique values for Postcode column
new_df = pd.DataFrame({'Postcode':wikidf.Postcode.unique()})
# Add text of Burough column to new dataframe
new_df['Borough']=pd.DataFrame(list(set(wikidf['Borough'].loc[wikidf['Postcode'] == pc['Postcode']])) for i, pc in new_df.iterrows())
# Iterates over the rows of the dataframe to add series of multiple Neighbourhoods in list into Neighbourhood column
new_df['Neighbourhood']=pd.Series(list(set(wikidf['Neighbourhood'].loc[wikidf['Postcode'] == pc['Postcode']])) for i, pc in new_df.iterrows())
# remove unwanted [] from text in Neighbourhood column of new dataframe
new_df['Neighbourhood']=new_df['Neighbourhood'].apply(lambda pc: ', '.join(pc))
#
new_df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [7]:
#3. d) If a cell has Borough but <<Not Assigned>> Neighbourhood, then Neighbourhood get same value as Borough
#
# For Boroughs with <<Not assigned>> Neighbourhood then Neighbourhood <<Not Assigned>> changed to Borough's value
new_df.Neighbourhood.replace("Not assigned", new_df.Borough, inplace=True)
#
new_df.head(12) # See M7A Queen's Park in row id #4

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
# 3. f) Show Number of (Rows, Columns) of new dataframe
# after testing table in Excel and removing duplicates, row count was 103
new_df.shape

(103, 3)